# IMPORT MODULE

In [154]:
import numpy as np
import matplotlib.pyplot as plt
import PIL as pil

# THAY ĐỔI ĐỘ SÁNG

In [155]:
def change_brightness(imageArray, percent):
    # Reshape lai hinh
    h, w, d = imageArray.shape
    a = imageArray.reshape(h * w, d)
    
    # Tao mang chua toan gia tri can cong vao hinh
    alphaArray = np.full((len(a), d), percent)
    newImageArray = a + alphaArray

    # kiem tra khoang [0,255]
    newImageArray[newImageArray > 255] = 255
    newImageArray[newImageArray < 0] = 0
    
    newImageArray = newImageArray.reshape(h,w,d)
    
    newImageArray = newImageArray.astype(np.uint8)
    return newImageArray
    

# THAY ĐỔI ĐỘ TƯƠNG PHẢN

In [156]:
def change_contrast(imageArray, contrast):
    # reshape lai anh
    h,w,d = imageArray.shape
    a = imageArray.reshape(h * w, d)

    # Tinh factor cho contrast
    factor = 131*(contrast + 127)/(127*(131-contrast))

    # Voi moi pixel, gan gia tri moi theo cong thuc
    newIMG = np.array(a, copy=1)
    newIMG = factor*(newIMG) + 127*(1-factor)
    
    # Check vuot qua khoang [0,255]
    newIMG[newIMG > 255] = 255
    newIMG[newIMG < 0] = 0

    newIMG = newIMG.reshape(h,w,d)
    newIMG = newIMG.astype(np.uint8)
    
    return newIMG

# LẬT ẢNH NGANG DỌC

In [157]:
def flip_image(image, type):
    if type == "horizon":
        reversed_arr = np.fliplr(image)
    elif type == "vertical":
        reversed_arr = np.flipud(image)

    reversed_arr = reversed_arr.astype(np.uint8)
    return reversed_arr

# CHUYỂN ĐỔI ẢNH RGB THÀNH ẢNH XÁM

In [158]:
def convert_to_grey(imageArray):
    # Tach rieng RGB cua tung pixel ra thanh 3 ma tran r,g,b rieng biet
    r, g, b = imageArray[:,:,0], imageArray[:,:,1], imageArray[:,:,2]

    # Tinh mau xam theo luminosity --> Duoc ma tran 1D mau xam tuong ung voi tat ca pixel cua hinh
    greyscale = r * 0.3 + g * 0.59 + b * 0.11

    greyscale = greyscale.astype(np.uint32)
    return greyscale

# CHỒNG 2 ẢNH CÙNG KÍCH THƯỚC

In [159]:
def stack_two_img(img1, img2):
    newIMG1 = convert_to_grey(img1)
    newIMG2 = convert_to_grey(img2)

    newIMG = newIMG1 + newIMG2

    newIMG = newIMG.astype(np.uint32)
    return newIMG

# LÀM MỜ ẢNH

In [160]:
def blur_img2(img, radius):
    a = np.array(img, copy=1)
    h,w,d = a.shape

    newIMG = np.array(a, copy=1)
    kernel = 2 * radius + 1

    replacePixel = (0,0,0)
    if d == 4:
        replacePixel = (0,0,0,255)

    # Xet pixel xung theo truc doc
    for x in range(0, h):
        for y in range(0, w):
            if x < radius or y < radius or x + radius >= h or y + radius >= w:
                newIMG[x][y] = replacePixel
                continue

            count = 0
            sum = replacePixel

            # Tim cac pixel xung quanh theo truc doc va add vao mang temp
            for i in range(x - radius, x - radius + kernel):
                    sum  += a[i][y - radius]
                    count += 1

            sum = sum *  (1 / count)
            newIMG[x][y] = sum
    
    # Xet pixel xung quanh theo truc theo truc ngang
    for x in range(0, h):
        for y in range(0, w):
            if x < radius or y < radius or x + radius >= h or y + radius >= w:
                newIMG[x][y] = replacePixel
                continue

            count = 0

            sum = replacePixel
            # Tim cac pixel xung quanh theo truc ngang va add vao mang temp
            for i in range(y - radius, y - radius + kernel):
                    sum += a[x - radius][i]
                    count += 1

            sum = sum *  (1 / count)
            newIMG[x][y] = sum
    
    newIMG = newIMG.astype(np.uint8)
    return newIMG

# CẮT KHUNG

In [161]:
def cut_image(img, type):
    a = np.array(img)
    h, w, d = a.shape
    centerPixel = (int(h/2), int(w/2))
    
    diagonal = np.sqrt([w ** 2 + h ** 2])
    diagonal = diagonal[0]

    replacePixel = (0,0,0)
    if d == 4:
        replacePixel = (0,0,0,255)

    if type == "cir":
        radius = int(min(h,w) / 2)
        for i in range(h):
            for j in range(w):
                if ((i - centerPixel[0]) ** 2) + ((j - centerPixel[1]) ** 2) > radius ** 2:
                    a[i][j] = replacePixel
    elif type == "ell":
        majorAxis  = diagonal / 2.5
        minorAxis = majorAxis / 2

        cosAngle = float((h / 2) / (diagonal / 2))
        sinAngle = float((w / 2) / (diagonal / 2))

        for i in range(h):
            for j in range(w):
                if ( ((cosAngle * (i - centerPixel[0]) + sinAngle * (j - centerPixel[1])) / majorAxis) ** 2) + (((cosAngle * (j - centerPixel[1]) - sinAngle * (i - centerPixel[0])) / minorAxis) ** 2) > 1:
                    if ( ((cosAngle * (i - centerPixel[0]) + (-sinAngle) * (j - centerPixel[1])) / majorAxis) ** 2) + (((cosAngle * (j - centerPixel[1]) - (-sinAngle) * (i - centerPixel[0])) / minorAxis) ** 2) > 1:
                        a[i][j] = replacePixel

    a = a.astype(np.uint8)
    return a


# MAIN

In [ ]:
def main():
    imgName1 = input("NHAP TEN ANH 1")
    img1 = pil.Image.open(imgName1)

    imgName = imgName1.split('.')
    imgName, extension = imgName[0], imgName[1]

    a = np.array(img1)        
    height, width, depth = a.shape
    
    print("0. Thuc hien het tat ca chuc nang")
    print("1. Thay doi do sang (co nhap vao do gia tri do sang)")
    print("2. Thay doi do tuong phan (co nhap vao gia tri tuong phan)")
    print("3. Lat anh ngang doc (nhap vao \"horizon\" hoac \"vertical\" ")
    print("4. Chuyen doi anh RGB thanh xam")
    print("5. Chong 2 anh mau xam (co nhap vao ten anh 2)")
    print("6. Lam mo anh (co nhap vao gia tri radius (kernel <-> 2 * radius + 1)")
    print("7. Cat anh theo khung (1. hinh tron, 2. ellipse)")

    choice = input("ENTER CHOICE")
    
    if choice == "0":
        choice2 = input("Goi het tat ca chuc nang Nhap lua chon cua ban: \n0-Su dung cac tham so default cho tat ca cac chuc nang\n1-Tu nhap cac tham so cho cac chuc nang")

        brightness, contrastVal, flipType = 100, 60, "vertical"
        radius, frameType = 1, "cir"

        if choice2 == "1":
            brightness = input("Nhap do sang cua anh")
            contrastVal = input("Nhap do tuong phan cua anh")
            flipType = input("Nhap loai lat (horizon - vertical)")
            radius = input("Nhap radius (kernel = 2 * radius + 1 with radius=0,1,2,3....)")
            frameType = input("Nhap loai khung (cir - ell)")

        # Chuan bi cho chuc nang xep chong 2 anh
        imgName2 = input("NHAP TEN ANH 2 cho chuc nang xep chong 2 anh")
        img2 = pil.Image.open(imgName2)
        b = np.array(img2)

        # thay doi do sang
        temp = change_brightness(a, brightness)
        plt.imsave(imgName + "_brightness." + extension, temp)

        # Thay doi do tuong phan
        temp = change_contrast(a, contrastVal)
        plt.imsave(imgName + "_contrast." + extension, temp)

        # Lat anh ngang doc
        temp = flip_image(img1, flipType)
        plt.imsave(imgName + "_flip_" + flipType + "." + extension, temp)

        # Chuyen RGB thanh xam
        temp = convert_to_grey(a)
        plt.imsave(imgName + "_greyscale." + extension, temp, cmap="gray")

        # Chong 2 anh mau xam
        temp = stack_two_img(a, b)
        plt.imsave(imgName + "_stack2IMG." + extension, temp, cmap="gray")

        # lam mo anh
        temp = blur_img2(a, radius)
        plt.imsave(imgName + "_blur." + extension, temp)

        # Dong khung
        temp = cut_image(a, frameType)
        plt.imsave(imgName + "_cutframe_" + frameType + "." + extension, temp)

        return

    # thay doi do sang
    if choice == "1":
        brightness = int(input("Nhap do sang cua anh"))
        a = change_brightness(a, brightness)
        plt.imsave(imgName + "_brightness." + extension, a)

    # Thay doi do tuong phan
    if choice == "2":
        contrastVal = int(input("Nhap do tuong phan cua anh"))
        a = change_contrast(a, contrastVal)
        plt.imsave(imgName + "_contrast." + extension, a)

    # lat anh ngang doc
    if choice == "3":
        flipType = input("Nhap loai lat (horizon - vertical)")
        a = flip_image(img1, flipType)
        plt.imsave(imgName + "_flip_" + flipType + "." + extension, a)

    # Chuyen thanh anh xam
    if choice == "4":
        a = convert_to_grey(a)
        plt.imsave(imgName + "_greyscale." + extension, a, cmap="gray")

    # stack 2 hinh
    if choice == "5":
        imgName2 = input("NHAP TEN ANH 2")
        img2 = pil.Image.open(imgName2)
        b = np.array(img2)

        a = stack_two_img(a, b)
        plt.imsave(imgName + "_stack2IMG." + extension, a, cmap="gray")

    # Lam mo hinh
    if choice == "6":
        radius = int(input("Nhap radius (kernel = 2 * radius + 1 with radius=0,1,2,3....)"))
        a = blur_img2(a, radius)
        plt.imsave(imgName + "_blur." + extension, a)

    # Cat khung hinh
    if choice == "7":
        frameType = input("Nhap loai khung (cir - ell)")
        a = cut_image(img1, frameType)
        plt.imsave(imgName + "_cutFrame_" + frameType + "." + extension, a)

    plt.imshow(a, cmap='gray')

main()